In [6]:
import pandas as pd
df = pd.read_csv(r"C:\Users\hp\Downloads\malicious_phish.csv\malicious_phish.csv") 

In [7]:
df.head()

,url,type
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
3,http://www.garage-pirenne.be/index.php?option=...,defacement
4,http://adventure-nicaragua.net/index.php?optio...,defacement


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651191 entries, 0 to 651190
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     651191 non-null  object
 1   type    651191 non-null  object
dtypes: object(2)
memory usage: 9.9+ MB


In [9]:
print(df.isnull().sum())

url     0
type    0
dtype: int64


In [10]:
df = df.drop_duplicates()

In [11]:
#Checking all urls are in lowercase
df['url'] = df['url'].str.lower()

# Strip spaces if any
df['url'] = df['url'].str.strip()


In [12]:
print(df['type'].value_counts())

benign        428080
defacement     95308
phishing       94092
malware        23645
Name: type, dtype: int64


In [14]:
#cleaning 
df['type'] = df['type'].str.lower().str.strip()

In [15]:
df['url_length'] = df['url'].str.len()
df['has_https'] = df['url'].str.startswith('https')

In [16]:
#renaming
df = df.rename(columns={'url': 'URL', 'type': 'Label'})

In [17]:
df.head()

,URL,Label,url_length,has_https
0,br-icloud.com.br,phishing,16,False
1,mp3raid.com/music/krizz_kaliko.html,benign,35,False
2,bopsecrets.org/rexroth/cr/1.htm,benign,31,False
3,http://www.garage-pirenne.be/index.php?option=...,defacement,88,False
4,http://adventure-nicaragua.net/index.php?optio...,defacement,235,False


In [18]:
#extracting 
from urllib.parse import urlparse

parsed = df['URL'].apply(urlparse)

df['domain'] = parsed.apply(lambda x: x.netloc)
df['path'] = parsed.apply(lambda x: x.path)
df['query'] = parsed.apply(lambda x: x.query)

In [19]:
df = df[df['URL'].notnull() & (df['URL'].str.strip() != "")]

In [20]:
from urllib.parse import urlparse

df['domain'] = df['URL'].apply(lambda x: urlparse(x).netloc)
df = df[df['domain'].str.contains('\.') & ~df['domain'].str.contains('localhost')]

In [22]:
df.head()

,URL,Label,url_length,has_https,domain,path,query
3,http://www.garage-pirenne.be/index.php?option=...,defacement,88,False,www.garage-pirenne.be,/index.php,option=com_content&view=article&id=70&vsig70_0=15
4,http://adventure-nicaragua.net/index.php?optio...,defacement,235,False,adventure-nicaragua.net,/index.php,option=com_mailto&tmpl=component&link=ahr0cdov...
5,http://buzzfil.net/m/show-art/ils-etaient-loin...,benign,118,False,buzzfil.net,/m/show-art/ils-etaient-loin-de-s-imaginer-que...,
8,http://www.pashminaonline.com/pure-pashminas,defacement,44,False,www.pashminaonline.com,/pure-pashminas,
11,http://www.ikenmijnkunst.nl/index.php/expositi...,defacement,64,False,www.ikenmijnkunst.nl,/index.php/exposities/exposities-2006,


In [24]:
df['url_length'] = df['URL'].str.len()
df = df[df['url_length'].between(10, 200)]  

In [ ]:
domain_counts = df['domain'].value_counts()
rare_domains = domain_counts[domain_counts < 10].index
df['domain'] = df['domain'].replace(rare_domains, 'other')

In [ ]:
df.to_csv('cleaned_url_dataset.csv', index=False)